In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
print("GPUs:", tf.config.list_physical_devices('GPU'))

!pip install csbdeep
!pip install aicsimageio
!pip install aicspylibczi

from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from aicsimageio import AICSImage
from csbdeep.utils import Path, download_and_extract_zip_file, plot_some
from csbdeep.io import save_tiff_imagej_compatible
from csbdeep.models import CARE

Mounted at /content/drive
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of ome-zarr to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.5 MB/s eta 0:00:00


In [ ]:


path = r'/content/drive/MyDrive/lattice_data/fixed/CARE/raw_data/b2-2a_2c_pos6-01-_raw_lifeact_T0.czi'

tif = False
czi = True

if tif:
  img = tiff.imread(path)
elif czi:
  img = AICSImage(path).data


print(img.shape)
img = img[0,0,...]
axes = 'ZYX'

(1, 1, 1658, 718, 2046)


In [ ]:

model = CARE(config=None, name='my_model', basedir='/content/drive/MyDrive/CARE/models', )

Loading network weights from 'weights_best.h5'.


In [ ]:
%%time
restored = model.predict(img, axes, n_tiles=(25,11,32))

100%|██████████| 9152/9152 [21:13<00:00,  7.19it/s]


CPU times: user 22min 20s, sys: 1min 4s, total: 23min 25s
Wall time: 22min 43s


In [ ]:
print(restored.shape)
import tifffile as tiff
import numpy as np


min_val = restored.min()
max_val = restored.max()

restored_norm = (restored - min_val) / (max_val - min_val)
restored_8bit = (65535 * np.clip(restored_norm, 0, 1)).astype(np.uint16)

tiff.imwrite('/content/drive/MyDrive/lattice_data/fixed/CARE/raw_data/for_training/outputs/timelapset0.tif' , restored_8bit)

In [ ]:
from tifffile import imread
import numpy as np
restored = imread(r'/content/drive/MyDrive/lattice_data/fixed/CARE/raw_data/for_training/outputs/whole_predict.tif')
print(restored.shape)

KeyboardInterrupt: 